In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, BatchNormalization, Activation, Dropout

def conv_block(tensor, n_filters, kernel_size=3, batchnorm=True):

    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), padding="same")(tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def deconv_block(tensor, residual, n_filters, kernel_size=3, strides=2, dropout=0.5):
    y = UpSampling2D(size=(strides, strides))(tensor)
    y = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), padding="same")(y)
    y = concatenate([y, residual], axis=3)
    y = Dropout(dropout)(y)
    y = conv_block(y, n_filters, kernel_size=3, batchnorm=True)

    return y

def unet_model(input_shape, n_filters=64, dropout=0.5, batchnorm=True):

    input_layer = Input(shape=input_shape)

    c1 = conv_block(input_layer, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D(pool_size=(2, 2))(c1)
    p1 = Dropout(dropout)(p1)

    c2 = conv_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D(pool_size=(2, 2))(c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D(pool_size=(2, 2))(c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    p4 = Dropout(dropout)(p4)

    c5 = conv_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)

    u6 = deconv_block(c5, residual=c4, n_filters=n_filters*8, kernel_size=3, strides=2, dropout=dropout)
    u7 = deconv_block(u6, residual=c3, n_filters=n_filters*4, kernel_size=3, strides=2, dropout=dropout)
    u8 = deconv_block(u7, residual=c2, n_filters=n_filters*2, kernel_size=3, strides=2, dropout=dropout)
    u9 = deconv_block(u8, residual=c1, n_filters=n_filters*1, kernel_size=3, strides=2, dropout=dropout)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(u9)

    model = Model(inputs=[input_layer], outputs=[outputs])

    return model



In [ ]:
import os
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Flatten, Activation, concatenate, ZeroPadding2D, BatchNormalization
from keras.optimizers import Adam
from keras.utils import to_categorical
import cv2
from tqdm import tqdm

batch_size = 32
epochs = 100

PREDICTION_SEGMENTATION_DIR = "prediction_segmentation"

if not os.path.exists(PREDICTION_SEGMENTATION_DIR):
    os.makedirs(PREDICTION_SEGMENTATION_DIR)

def read_lines(flist_name, shuffle=True):

    with open(flist_name, 'r') as f:
        lines = f.readlines()
        if shuffle:
            np.random.shuffle(lines)
    return lines

def save_segmentation_masks(model, directory):
    all_files = os.listdir(directory)
    for file in tqdm(all_files):
        img = cv2.imread(os.path.join(directory, file))
        img = cv2.resize(img, (INPUT_SIZE[0], INPUT_SIZE[1]))
        pred = model.predict(np.expand_dims(img, axis=0))
        mask = np.argmax(pred, axis=-1)
        mask = np.squeeze(mask, axis=0)
        cv2.imwrite(os.path.join(PREDICTION_SEGMENTATION_DIR, file), mask*255)

if __name__ == "__main__":

    for fold_no in range(0, 5):
        model = network_unet(input_shape=(INPUT_SIZE[0], INPUT_SIZE[1], 3))
        optimizer_name = Adam(lr=1e-4)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer_name, metrics=['accuracy'])

        for ep in range(epochs):
          model.fit(X_train, y_train, batch_size=32, validation_split=0.1)
          model_path = os.path.join(root_dir, f'u_net_fold_{fold_no}.h5')
          model.save(model_path)


    test_directory = "data_kaggle/test"
    save_segmentation_masks(model, test_directory)


Test 1

